In [15]:
import os
from multiprocessing import Pool
from itertools import combinations

import datetime
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

from catboost import CatBoostClassifier
from catboost import CatBoost
from catboost import Pool
from catboost.text_processing import Tokenizer, Dictionary

%matplotlib inline

In [41]:
train = pd.read_csv('../data/lemmatized_train.csv')
train.head()

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad,lemmatized_desc
0,Диван-кровать,Продаем диван-кровать. Удобный механизм - евро...,Мебель и интерьер,Для дома и дачи,7000.0,Россия,Москва,2019-06-01 00:00:15.180656,0,продавать диван - кровать . удобный механизм -...
1,Кожух рулевой колонки Даф хф 91 4509834,Кожух рулевой колонки DAF XF 94 (60066004)/\n ...,Запчасти и аксессуары,Транспорт,2290.0,Россия,Москва,2019-06-01 00:00:44.317933,0,кожух рулевой колонка daf xf 94 ( 60066004)/ \...
2,Дешёвый буст аккаунтов Dota 4,! Буст аккаунтов с ммр выше 1000ммр не беру ! ...,Предложение услуг,Услуги,200.0,Северная Осетия,Владикавказ,2019-06-01 00:00:50.249692,1,! буст аккаунт с ммр выше 1000ммр не брать ! /...
3,Телевизор sharp.Смарт тв.Интернет,Продам телевизор . Диагональ 450.наличие входа...,Аудио и видео,Бытовая электроника,25000.0,Калининградская область,Советск,2019-06-01 00:00:50.325799,1,продам телевизор . диагональ 450.наличие вход ...
4,Открытка-конверт,Открытки-конверты ручной работы/\nВыполнены в ...,Коллекционирование,Хобби и отдых,150.0,Ставропольский край,Ессентукская,2019-06-01 00:00:56.632655,0,открытка - конверты ручной работы/ \n выполнен...


In [42]:
train['telegram'] = train['description'].str.contains('[Tt][eE][lL][eE][gG][Rr][Aa][mM]')
train['telegram_2'] = train['description'].str.contains('[Tt][eE][lL][eE][gG][Rr][Aa][mM]\s*:?\s*[a-zA-Z\d]')
train['telegram_3'] = train['description'].str.contains('[Tt][gG]\s*:?\s*[a-zA-Z\d]')
train['num_1'] = train['description'].str.contains('\+[\s]*7')
train['num_2'] = train['description'].str.contains('8[-\s]*9')
train['num_3'] = train['description'].str.contains('8[-\s]*\(\s*9')
train['vk_dot_com'] = train['description'].str.contains('[vV][kK]\.com')
train['vk_dot_com_2'] = train['description'].str.contains('[vV][kK]\.com/[a-zA-Z\d]')
train['instagram'] = train['description'].str.contains('[Ii][nN][sS][tT]')
train['instagram_2'] = train['description'].str.contains('[Ii][nN][sS][tT]\s*:')
train['instagram_3'] = train['description'].str.contains('[Ii][nN][sS][tT]\s*:\s*[a-zA-Z\d]')

In [43]:
val = pd.read_csv('../data/lemmatized_val.csv')
val.head()

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad,lemmatized_desc
0,Шины,Звонить 89425546881,Запчасти и аксессуары,Транспорт,2000.0,Тульская область,Огаревка,2019-10-10 00:00:25.200714,1,звонить 89425546881
1,Продается мобильная перегородка с дверью,"Мобильная перегородка, предназначена для разгр...",Оборудование для бизнеса,Для бизнеса,10500.0,Вологодская область,Вологда,2019-10-10 00:03:11.527292,0,"мобильный перегородка , предназначить для разг..."
2,Комплект зимних шин на дисках 682/32/64,Шины зимние б/у Marshal Assimetric I”Zen KW 61...,Запчасти и аксессуары,Транспорт,4000.0,Россия,Москва,2019-10-10 00:05:07.193248,1,шина зимний б/у marshal assimetric i”zen kw 61...
3,Кровать-трансормер Дакота сб-4085,"Продаю кровать-трансформер производства ""Столп...",Мебель и интерьер,Для дома и дачи,17000.0,Московская область,Химки,2019-10-10 00:05:58.165179,0,"продавать кровать - трансформер производство ""..."
4,Honda VFR 800 2004 г.в,"Мот в отличном состоянии для своих лет, Родной...",Мотоциклы и мототехника,Транспорт,235000.0,Брянская область,Брянск,2019-10-10 00:06:19.231151,0,"мот в отличный состояние для свой год , родной..."


In [44]:
val['telegram'] = val['description'].str.contains('[Tt][eE][lL][eE][gG][Rr][Aa][mM]')
val['telegram_2'] = val['description'].str.contains('[Tt][eE][lL][eE][gG][Rr][Aa][mM]\s*:?\s*[a-zA-Z\d]')
val['telegram_3'] = val['description'].str.contains('[Tt][gG]\s*:?\s*[a-zA-Z\d]')
val['num_1'] = val['description'].str.contains('\+[\s]*7')
val['num_2'] = val['description'].str.contains('8[-\s]*9')
val['num_3'] = val['description'].str.contains('8[-\s]*\(\s*9')
val['vk_dot_com'] = val['description'].str.contains('[vV][kK]\.com')
val['vk_dot_com_2'] = val['description'].str.contains('[vV][kK]\.com/[a-zA-Z\d]')
val['instagram'] = val['description'].str.contains('[Ii][nN][sS][tT]')
val['instagram_2'] = val['description'].str.contains('[Ii][nN][sS][tT]\s*:')
val['instagram_3'] = val['description'].str.contains('[Ii][nN][sS][tT]\s*:\s*[a-zA-Z\d]')

In [24]:
test.columns

Index(['title', 'description', 'subcategory', 'category', 'price', 'region',
       'city', 'datetime_submitted', 'is_bad', 'lemmatized_desc', 'telegram',
       'num_1', 'num_2', 'vk_dot_com', 'instagram'],
      dtype='object')

In [52]:
features = ['price', 'telegram', 'telegram_2', 'telegram_3', 'num_1', 'num_2', 'num_3', 
            'vk_dot_com', 'vk_dot_com_2', 'instagram', 'instagram_2', 'instagram_3']
cat_features = ['category', 'city']
text_features = ['lemmatized_desc']

In [49]:
tokenizer = Tokenizer(lowercasing=True, 
                      separator_type='BySense', 
                      token_types=['Word', 'Number', 'Punctuation'])
dictionary = Dictionary(occurence_lower_bound=500,
                        max_dictionary_size=15000,
                        num_bpe_units=2500,
                        dictionary_type='Bpe')

In [53]:
text_processing = {
    "tokenizers" : [{
        "tokenizer_id" : "tokenizer",
        'lowercasing' : 'True', 
        'separator_type' : 'BySense', 
        'token_types' : ['Word', 'Number', 'Punctuation']
    }],

    "dictionaries" : [{
        "dictionary_id" : "dictionary",
        'occurence_lower_bound' : '500',
        'max_dictionary_size' : '15000',
        'num_bpe_units' : '2500',
        'dictionary_type' : 'Bpe'
    }],

    "feature_processing" : {
        "default" : [{
            "dictionaries_names" : ["dictionary"],
            "feature_calcers" : ["BoW"],
            "tokenizers_names" : ["tokenizer"]
        }]
    }
}

In [54]:
train_pool = Pool(train[features + cat_features],
                  label=train['is_bad'],
                  cat_features=cat_features)
test_pool = Pool(val[features + cat_features],
                 label=val['is_bad'],
                 cat_features=cat_features)

In [55]:
clf = CatBoostClassifier(iterations=2500, 
                         verbose=100, 
                         thread_count=2, 
                         depth=5,
                         l2_leaf_reg=1,
                         text_processing=text_processing)

In [56]:
clf.fit(train_pool)

Learning rate set to 0.044677
0:	learn: 0.6598392	total: 980ms	remaining: 1h 21m 39s
30:	learn: 0.3728185	total: 14.8s	remaining: 39m 34s
60:	learn: 0.3501227	total: 30.5s	remaining: 41m 5s
90:	learn: 0.3432791	total: 49.5s	remaining: 44m 31s
120:	learn: 0.3403056	total: 1m 13s	remaining: 49m 6s
150:	learn: 0.3385768	total: 1m 34s	remaining: 50m 42s
180:	learn: 0.3373024	total: 1m 52s	remaining: 50m 1s
210:	learn: 0.3361696	total: 2m 9s	remaining: 48m 51s
240:	learn: 0.3354610	total: 2m 27s	remaining: 48m 26s
270:	learn: 0.3347348	total: 2m 45s	remaining: 48m 11s
300:	learn: 0.3340408	total: 3m 11s	remaining: 49m 44s
330:	learn: 0.3334706	total: 3m 28s	remaining: 48m 59s
360:	learn: 0.3329834	total: 3m 44s	remaining: 48m 1s
390:	learn: 0.3325510	total: 4m 1s	remaining: 47m 26s
420:	learn: 0.3321707	total: 4m 18s	remaining: 46m 52s
450:	learn: 0.3318833	total: 4m 34s	remaining: 46m 10s
480:	learn: 0.3315476	total: 4m 51s	remaining: 45m 38s
510:	learn: 0.3312538	total: 5m 11s	remaining: 

In [57]:
y_pred = clf.predict_proba(test_pool, thread_count=2)

In [58]:
roc_auc_score(val['is_bad'], y_pred[:, 1])

0.8432392309947445

In [60]:
scores = []
for category in val['category'].unique():
    scores.append(
        roc_auc_score(
            val['is_bad'][val['category'] == category], 
            np.clip(0, 1, y_pred[val['category'] == category][:, 1] + val[val['category'] == category]['num_1'])
        )
    )
np.mean(scores)

0.7276548248402579

In [61]:
scores

[0.8813720598772427,
 0.7240111147433803,
 0.7027182221628521,
 0.617635197874518,
 0.6438305528587079,
 0.6420650209284627,
 0.8848454933949887,
 0.7573491670943959,
 0.6846715328467152,
 0.7380498866213153]

In [83]:
def sigmoid(x, w1=0.5, w2=1.1797474):
    return (1 / (1 + np.exp(-x[0] * w1) * np.exp(-x[1] * w2)))

In [84]:
sigmoid([1, 1])

0.842871079813761

In [94]:
1 / (1 + np.exp(-0) * np.exp(-0))

0.5